In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, Activation, MaxPooling2D ,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import regularizers

def create_alexnet(in_shape=(227, 227, 3), n_classes=10, kernel_regular=None):
    input_tensor = Input(shape=in_shape)
    
    x = Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid')(input_tensor)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Flatten()(x)
    
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output = Dense(units=n_classes, activation='softmax')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    
    return model

In [8]:
model = create_alexnet(in_shape=(227, 227, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_5 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_6 (Activation)    (None, 27, 27, 256)       0   

In [9]:
from tensorflow.keras.datasets import cifar10

# 전체 6만개 데이터 중, 5만개는 학습 데이터용, 1만개는 테스트 데이터용으로 분리
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

train dataset shape: (50000, 32, 32, 3) (50000, 1)
test dataset shape: (10000, 32, 32, 3) (10000, 1)


In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd

import random as python_random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10

def zero_one_scaler(image):
    return image/255.0

def get_preprocessed_ohe(images, labels, pre_func=None):
    # preprocessing 함수가 입력되면 이를 이용하여 image array를 scaling 적용.
    if pre_func is not None:
        images = pre_func(images)
    # OHE 적용    
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels )

# CIFAR10 데이터 재 로딩 및 Scaling/OHE 전처리 적용하여 학습/검증/데이터 세트 생성. 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)
(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)

print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(40000, 32, 32, 3) (40000, 10) (10000, 32, 32, 3) (10000, 10) (10000, 32, 32, 3) (10000, 10)


In [12]:
model = create_alexnet(in_shape=(128, 128, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 30, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 30, 30, 96)        384       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 256)       614656    
_________________________________________________________________
activation_11 (Activation)   (None, 14, 14, 256)       0   

In [13]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

In [17]:
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        
        if self.shuffle:
            #self.on_epoch_end()
            pass
        
    def __len__(self):
        return int(np.ceil(len(self.labels)/self.batch_size))
    
    def __getitem__(self, index):
        image_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            
        image_batch = np.zeros((len(image_fetch), IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        for image_index in range(len(image_fetch)):
            image = cv2.resize(image_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))
            
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
                
            if self.pre_func is not None:
                image = self.pre_func(image)
                
            image_batch[image_index] = image
            
        return image_batch, label_batch
    
    def __on_epoch_end(self):
        if (self.shuffle):
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass

In [21]:
tr_ds = CIFAR_Dataset(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=True, pre_func=zero_one_scaler)
val_ds = CIFAR_Dataset(val_images, val_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=zero_one_scaler)

In [22]:
next(iter(tr_ds))[0].shape, next(iter(val_ds))[0].shape, next(iter(tr_ds))[1].shape, next(iter(val_ds))[1].shape, next(iter(tr_ds))[0][0]

((64, 128, 128, 3),
 (64, 128, 128, 3),
 (64, 10),
 (64, 10),
 array([[[0.6431373 , 0.68235296, 0.69411767],
         [0.6431373 , 0.68235296, 0.69411767],
         [0.63529414, 0.67058825, 0.68235296],
         ...,
         [0.28627452, 0.2901961 , 0.27058825],
         [0.27450982, 0.28235295, 0.26666668],
         [0.27450982, 0.28235295, 0.26666668]],
 
        [[0.6431373 , 0.68235296, 0.69411767],
         [0.6431373 , 0.68235296, 0.69411767],
         [0.63529414, 0.67058825, 0.68235296],
         ...,
         [0.28627452, 0.2901961 , 0.27058825],
         [0.27450982, 0.28235295, 0.26666668],
         [0.27450982, 0.28235295, 0.26666668]],
 
        [[0.6509804 , 0.6901961 , 0.7019608 ],
         [0.6509804 , 0.6901961 , 0.7019608 ],
         [0.6431373 , 0.6784314 , 0.6901961 ],
         ...,
         [0.28627452, 0.2901961 , 0.27058825],
         [0.2784314 , 0.28235295, 0.26666668],
         [0.2784314 , 0.28235295, 0.26666668]],
 
        ...,
 
        [[0.9137255 , 0.85

In [23]:
model = create_alexnet(in_shape=(128, 128, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

history = model.fit(tr_ds, epochs=30, validation_data=val_ds, callbacks=[rlr_cb, ely_cb])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 30, 30, 96)        34944     
_________________________________________________________________
activation_20 (Activation)   (None, 30, 30, 96)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 30, 30, 96)        384       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 14, 256)       614656    
_________________________________________________________________
activation_21 (Activation)   (None, 14, 14, 256)       0   

In [24]:
test_ds = CIFAR_Dataset(test_images, test_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=zero_one_scaler)
model.evaluate(test_ds)

157/157 [==============================] - 3s 19ms/step - loss: 2.2558 - accuracy: 0.8033


[2.255783796310425, 0.8033000230789185]